# Day 7

While snooping around the local network of EBHQ, you compile a list of IP addresses (they're IPv7, of course; IPv6 is much too limited). You'd like to figure out which IPs support TLS (transport-layer snooping).

An IP supports TLS if it has an Autonomous Bridge Bypass Annotation, or ABBA. An ABBA is any four-character sequence which consists of a pair of two different characters followed by the reverse of that pair, such as xyyx or abba. However, the IP also must not have an ABBA within any hypernet sequences, which are contained by square brackets.

For example:

- `abba[mnop]qrst` supports TLS (abba outside square brackets).
- `abcd[bddb]xyyx` does not support TLS (bddb is within square brackets, even though xyyx is outside square brackets).
- `aaaa[qwer]tyui` does not support TLS (aaaa is invalid; the interior characters must be different).
- `ioxxoj[asdfgh]zxcvbn` supports TLS (oxxo is outside square brackets, even though it's within a larger string).

How many IPs in your puzzle input support TLS?



In [170]:
import re
from pprint import pprint

In [94]:
hypernet_pattern = re.compile(r'\[(\w+)\]')

In [101]:
hypernet_pattern.findall("abba[mnop]qrst")

['mnop']

In [102]:
hypernet_pattern.findall("gdlrknrmexvaypu[crqappbbcaplkkzb]vhvkjyadjsryysvj[nbvypeadikilcwg]jwxlimrgakadpxu[dgoanojvdvwfabtt]yqsalmulblolkgsheo")

['crqappbbcaplkkzb', 'nbvypeadikilcwg', 'dgoanojvdvwfabtt']

In [103]:
abba_pattern = re.compile(r'(\w)(?!\1)(\w)\2\1')

In [104]:
# this should work
abba_pattern.search('cabba').group()

'abba'

In [105]:
# this should fail
abba_pattern.search('caaaa').group()

AttributeError: 'NoneType' object has no attribute 'group'

In [140]:
def supports_tls(ip):
    # First find hypernet bits, check they aren't ABBA
    hypernets = hypernet_pattern.findall(ip)
    for h in hypernets:
        if abba_pattern.search(h) is not None:
            return False

    # Now go through everything else to find an ABBA
    # Make a custom regex for this particular IP
    other_parts_str = r'(\w+)'
    other_parts_str += r'(\w+)'.join([r'\['+h+r'\]' for h in hypernets])
    other_parts_str += r'(\w+)'
    other_pattern = re.compile(other_parts_str)
    m = other_pattern.match(ip)
    for a in m.groups():
        if abba_pattern.search(a):
            return True
    return False

In [128]:
supports_tls("abba[mnop]qrst")

True

In [129]:
supports_tls("abcd[bddb]xyyx")

False

In [130]:
supports_tls("aaaa[qwer]tyui")

False

In [131]:
supports_tls("ioxxoj[asdfgh]zxcvbn")

True

In [132]:
supports_tls("gdlrknrmexvaypu[crqappbbcaplkkzb]vhvkjyadjsryysvj[nbvypeadikilcwg]jwxlimrgakadpxu[dgoanojvdvwfabtt]yqsalmulblolkgsheo")

False

In [133]:
with open("Day_7_input.txt") as f:
    answer = len([x for x in f.readlines() if supports_tls(x.strip())])
    print(answer, "matches")


110 matches


## Part 2

You would also like to know which IPs support SSL (super-secret listening).

An IP supports SSL if it has an Area-Broadcast Accessor, or ABA, anywhere in the supernet sequences (outside any square bracketed sections), and a corresponding Byte Allocation Block, or BAB, anywhere in the hypernet sequences. An ABA is any three-character sequence which consists of the same character twice with a different character between them, such as xyx or aba. A corresponding BAB is the same characters but in reversed positions: yxy and bab, respectively.

For example:

- `aba[bab]xyz` supports SSL (aba outside square brackets with corresponding bab within square brackets).
- `xyx[xyx]xyx` does not support SSL (xyx, but no corresponding yxy).
- `aaa[kek]eke` supports SSL (eke in supernet with corresponding kek in hypernet; the aaa sequence is not related, because the interior character must be different).
- `zazbz[bzb]cdb` supports SSL (zaz has no corresponding aza, but zbz has a corresponding bzb, even though zaz and zbz overlap).

How many IPs in your puzzle input support SSL?



In [264]:
# need the ?= to lookahaead to catch overlappign patterns, eg srsr
aba_pattern = re.compile(r'(?=(\w)(?!\1)(\w)\1)') 

In [265]:
aba_pattern.search("cdcdc").group()

''

In [266]:
# returns groups, so (\1, \2)
aba_pattern.findall("cdcdc")

[('c', 'd'), ('d', 'c'), ('c', 'd')]

In [255]:
def supports_ssl(ip):
    # First find hypernet bits, create corresponding anti patterns
    hypernets = hypernet_pattern.findall(ip)
    hypernets_babs = []
    for h in hypernets:
        for m in aba_pattern.finditer(h):
            if m:
                hypernets_babs.append(m.expand(r'\1\2\1'))
           
    aba_patterns = [re.compile('{0}{1}{0}'.format(x[1], x[0])) for x in hypernets_babs]
    
    # Now go through everything else to find an ABA
    # Make a custom regex for this particular IP
    other_parts_str = r'(\w+)'
    other_parts_str += r'(\w+)'.join([r'\['+h+r'\]' for h in hypernets])
    other_parts_str += r'(\w+)'
    other_pattern = re.compile(other_parts_str)
    m = other_pattern.match(ip)
    for a in m.groups():    
        if any(pattern.search(a) for pattern in aba_patterns):
            return True
    return False

In [256]:
supports_ssl("aba[bab]xyz")

True

In [257]:
supports_ssl("xyx[xyx]xyx")

False

In [258]:
supports_ssl("aaa[kek]eke")

True

In [259]:
supports_ssl("zazbz[bzb]cdb")

True

In [260]:
supports_ssl("gdlrknrmexvaypu[crqappbbcaplkkzb]vhvkjyadjsryysvj[nbvypeadikilcwg]jwxlimrgakadpxu[dgoanojvdvwfabtt]yqsalmulblolkgsheo")

False

In [261]:
supports_ssl("cfurhuhjrbxqoefybl[kdcazzlfacaurqguqkz]yufsmycojcxiiomwteo[zcnzchsersrsapze]bhkpjaybdyilwdomfr[ddbxqanevcpjuodnj]ttmxojmazfqzqxlz")

False

In [263]:
with open("Day_7_input.txt") as f:
    matches = [x.strip() for x in f if supports_ssl(x.strip())]
    print(len(matches), "matches")
    pprint(matches)


242 matches
['zlqdqmuxebccmndzbl[ykefimjzdqdmfvlflj]ptlphteflzxmolkof',
 'igefoemugshofmibco[uhahihzaglmzdpzjvfp]tfbuuhoughgismec[inbtuzxnxekfkulodyk]fxykxfkfnjvswwc',
 'mbweohfcgybqcqnl[yafsvfrduertfqze]hqaodhzkmhzmlrxuc[bytcgnvzvoovirqwn]njivpwgkkqvgowpenh[erodavzscuubhea]gizvzrqjzhkikhb',
 'azrzthfimarcdbk[usfjkmhedaqpfnisek]yqowqlqvlranjjvbauq[korlrbzcgrneashdrrq]fjicirnofvlrlnnkeqb[ktlfmzrqxcntvasev]urpuwoiogtcwskygxz',
 'qjmlxlnqzipdflotzl[mwewadvcvkoqjlvlruk]aciqxygnygyordpcvc[sirhqhrjopudmfub]kxexybjqhmqmukxmpug[bbvccqpfdebfmnvald]lgqcpzwrjzlhdcalvxh',
 'snwnqixjgwhcrpfeun[mvseymbltdzywnw]xbogzgtddtzzadgsrin[sibgoazaxuyfaaf]tdtrrjbxjzusuvzogpa[etytgiqwoyxevcq]ifanoaaqoldczzj',
 'laqkpspaxfqdpnrr[sskaqoytvzoxubh]viaivwettzucesoz[lfyzcvouvgaiavpxnqt]lqoedmocedvtgehdeok[gceosyhfjabmrgdhyve]fstmrfbyesmoeuzjzd',
 'ubmtbxlcwzyjxkq[bdcpucmtupuahyhrg]qbacwmgpwrgwehhcley[vjrphfuixndpcqvlrx]myyojfenvnbulfpfvm[ebskbwkmpfpicpii]jqcknedxkvofewgb',
 'bcocswwxbpcrscral[zrffpdwqlffhxdbocop]fsn